In [11]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np
from keras.layers import Embedding, LSTM, Dense, Dropout


In [12]:
data = """The cat and her kittens
They put on their mittens,
To eat a Christmas pie.
The poor little kittens
They lost their mittens,
And then they began to cry.
O mother dear, we sadly fear
We cannot go to-day,
For we have lost our mittens."
"If it be so, ye shall not go,
For ye are naughty kittens."""

In [13]:
"""tokenizer = Tokenizer(char_level=True)
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
print(tokenizer.word_index)
total_words = len(tokenizer.word_index) + 1
input_sequences = []
for line in corpus :
    token_list = tokenizer.texts_to_sequences([line])[0]
    #print(token_list)
    for i in range(1, len(token_list)) :
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence) 
#print(input_sequences)
max_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_len, padding='pre'))
predictors, label = input_sequences[:,:-1],input_sequences[:,-1] # predictors all rows and in each row leave the last. label- all rows with last one in each row
label = ku.to_categorical(label, num_classes=total_words)"""

'tokenizer = Tokenizer(char_level=True)\ncorpus = data.lower().split("\n")\ntokenizer.fit_on_texts(corpus)\nprint(tokenizer.word_index)\ntotal_words = len(tokenizer.word_index) + 1\ninput_sequences = []\nfor line in corpus :\n    token_list = tokenizer.texts_to_sequences([line])[0]\n    #print(token_list)\n    for i in range(1, len(token_list)) :\n        n_gram_sequence = token_list[:i+1]\n        input_sequences.append(n_gram_sequence) \n#print(input_sequences)\nmax_len = max([len(x) for x in input_sequences])\ninput_sequences = np.array(pad_sequences(input_sequences, maxlen=max_len, padding=\'pre\'))\npredictors, label = input_sequences[:,:-1],input_sequences[:,-1] # predictors all rows and in each row leave the last. label- all rows with last one in each row\nlabel = ku.to_categorical(label, num_classes=total_words)'

In [14]:
max_len

8

In [15]:
len({'x':2, y:3})

NameError: name 'y' is not defined

In [16]:
#Prepare the data
tokenizer = Tokenizer()
def dataset_preparation(data):
    """
    This function is to prepare input data to language model. 
    
    Parameters :
    data : 'text ie str format'
    
    Returns : 
    input data to model.
    """
    corpus = data.lower().split("\n") # list of strings
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1 # number of words
    input_sequences = [] 
    for line in corpus :
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)) :
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence) # 
    print(input_sequences)
    max_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_len, padding='pre'))
    predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_len, total_words


In [17]:
def create_model(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    
    #Build a seq lstm keras model
    model = Sequential()
    model.add(Embedding(input_dim=total_words, output_dim=10, input_length=input_len))
    model.add(LSTM(150))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
    model.fit(predictors, label, epochs=100, verbose=1, callbacks=[earlystop])
    print(model.summary())
    return model 
    

In [28]:
def generate_text(seed_text, next_words, max_sequence_len, model):
    for j in range(next_words) :
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items() :
            if index == predicted :
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text
                

In [19]:
predictors, label, max_len, total_words = dataset_preparation(data)
model = create_model(predictors, label, max_len, total_words)


[[6, 13], [6, 13, 7], [6, 13, 7, 14], [6, 13, 7, 14, 1], [2, 15], [2, 15, 16], [2, 15, 16, 8], [2, 15, 16, 8, 3], [4, 17], [4, 17, 18], [4, 17, 18, 19], [4, 17, 18, 19, 20], [6, 21], [6, 21, 22], [6, 21, 22, 1], [2, 9], [2, 9, 8], [2, 9, 8, 3], [7, 23], [7, 23, 2], [7, 23, 2, 24], [7, 23, 2, 24, 4], [7, 23, 2, 24, 4, 25], [26, 27], [26, 27, 28], [26, 27, 28, 5], [26, 27, 28, 5, 29], [26, 27, 28, 5, 29, 30], [5, 31], [5, 31, 10], [5, 31, 10, 4], [5, 31, 10, 4, 32], [11, 5], [11, 5, 33], [11, 5, 33, 9], [11, 5, 33, 9, 34], [11, 5, 33, 9, 34, 3], [35, 36], [35, 36, 37], [35, 36, 37, 38], [35, 36, 37, 38, 12], [35, 36, 37, 38, 12, 39], [35, 36, 37, 38, 12, 39, 40], [35, 36, 37, 38, 12, 39, 40, 10], [11, 12], [11, 12, 41], [11, 12, 41, 42], [11, 12, 41, 42, 1]]
Epoch 1/100
48/48 [==============================] - 2s 48ms/step - loss: 3.7617
Epoch 2/100
48/48 [==============================] - 0s 788us/step - loss: 3.7580
Epoch 3/100
48/48 [==============================] - 0s 703us/step - l

C:\Users\Shashank\Anaconda3\lib\site-packages\keras\callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


48/48 [==============================] - 0s 513us/step - loss: 3.7353
Epoch 9/100
48/48 [==============================] - 0s 484us/step - loss: 3.7311
Epoch 10/100
48/48 [==============================] - 0s 409us/step - loss: 3.7235
Epoch 11/100
48/48 [==============================] - 0s 570us/step - loss: 3.7105
Epoch 12/100
48/48 [==============================] - 0s 571us/step - loss: 3.7013
Epoch 13/100
48/48 [==============================] - 0s 539us/step - loss: 3.6812
Epoch 14/100
48/48 [==============================] - 0s 494us/step - loss: 3.6627
Epoch 15/100
48/48 [==============================] - 0s 494us/step - loss: 3.6571
Epoch 16/100
48/48 [==============================] - 0s 574us/step - loss: 3.6217
Epoch 17/100
48/48 [==============================] - 0s 548us/step - loss: 3.6039
Epoch 18/100
48/48 [==============================] - 0s 557us/step - loss: 3.5717
Epoch 19/100
48/48 [==============================] - 0s 512us/step - loss: 3.5948
Epoch 20/100
48/48

In [53]:
data

'The cat and her kittens\nThey put on their mittens,\nTo eat a Christmas pie.\nThe poor little kittens\nThey lost their mittens,\nAnd then they began to cry.\nO mother dear, we sadly fear\nWe cannot go to-day,\nFor we have lost our mittens."\n"If it be so, ye shall not go,\nFor ye are naughty kittens.'

In [30]:
print(generate_text("we sadly", 4, max_len, model))


we sadly we to day fear
